In [ ]:
import re
import pandas as pd

In [ ]:
f = open('demo_chat.txt', 'r', encoding='utf-8')

In [ ]:
data = f.read()

In [ ]:
print(data)

In [ ]:
data = re.sub(r'(\d+:\d+)\s*([ap]m)', r'\1', data)

In [ ]:
print(data)

In [ ]:
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
messages = re.split(pattern, data)[1:]

messages

In [ ]:
dates = re.findall(pattern, data)
dates

In [ ]:
df = pd.DataFrame({'user_message':messages, 'message_date': dates})
# convert message_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')
df.rename(columns={'message_date': 'date'}, inplace=True)

df.head()


In [ ]:
df.shape

In [ ]:
# separate users and messages
users = []
messages = []

for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:# user name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
        
        
df['user'] = users
df ['message'] = messages
df.drop(columns=['user_message'], inplace=True)

In [ ]:
df.head(5)

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df.head()

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df.head()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df.head()

In [ ]:
df['hour'] = df['date']. dt. hour
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
x =df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round(df['user'].value_counts()/df.shape[0]*100,2).reset_index().rename(columns={'index': 'name', 'user': 'percent'})

In [ ]:
temp = df[df['user']!= 'group_notification']
temp = temp[temp['message'] != '<Media omitted>\n']

In [ ]:
f = open('stop_hinglish.txt', 'r')
stop_words = f.read()
print(stop_words)

In [ ]:
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
words = []
for message in df ['message']:
         words.extend (message.split ())

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
df['month_num'] = df ['date'].dt.month

In [ ]:
timeline = df.groupby(['year','month_num', 'month']).count()['message'].reset_index()

In [ ]:
timeline

In [ ]:
time = []
for i in range(timeline. shape[0]) :
    time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))

In [ ]:
timeline['time'] = time

In [ ]:
timeline

In [ ]:
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df['only_date'] = df ['date'].dt.date

In [ ]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'], daily_timeline['message'])

In [ ]:
df['day_name'] = df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()